<a href="https://colab.research.google.com/github/mars241/NLP_Natural-Language-Processing/blob/main/Corrig%C3%A9_de_Nicolas_NLP_4_Bag_of_Words.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
# DANS GOOGLE DRIVE (A LA RACINE)

from google.colab import files
data_to_load = files.upload()

Saving train.csv to train.csv


In [ ]:
df = pd.read_csv("train.csv", sep=",", low_memory=False)

In [ ]:
len(df)

27480

In [ ]:
df.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27480 entries, 0 to 27479
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   textID         27480 non-null  object
 1   text           27480 non-null  object
 2   selected_text  27480 non-null  object
 3   sentiment      27480 non-null  object
dtypes: object(4)
memory usage: 858.9+ KB


In [ ]:
df.shape

(27480, 4)

# Ne garde que les tweets positifs et négatifs (tu excluras donc les neutral). Quel est le pourcentage de tweets positifs/négatifs ?

In [ ]:
df['sentiment'].unique()

array(['neutral', 'negative', 'positive'], dtype=object)

In [ ]:
df_pos_neg = df[df['sentiment'] != 'neutral']

In [ ]:
df_pos_neg.head()

,textID,text,selected_text,sentiment
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative
6,6e0c6d75b1,2am feedings for the baby are fun when he is a...,fun,positive


In [ ]:
tweet_neg_pct = df_pos_neg[df_pos_neg['sentiment'] != 'negative']['sentiment'].count() / len(df_pos_neg)*100
tweet_pos_pct = df_pos_neg[df_pos_neg['sentiment'] != 'positive']['sentiment'].count() / len(df_pos_neg)*100

In [ ]:
print(f'Il y a {round(tweet_neg_pct,1)} % de tweets négatifs et {round(tweet_pos_pct,1)} % de tweets positifs')

Il y a 52.4 % de tweets négatifs et 47.6 % de tweets positifs


# Copie la colonne text dans une Série X, et la colonne sentiment dans une Série y. Applique un train test split avec le random_state = 32 et un train_size de 0.75.

In [ ]:
X = df_pos_neg['selected_text']
y = df_pos_neg['sentiment']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state=32)

# Crée un modèle vectorizer avec scikit-learn en utilisant la méthode Countvectorizer. Entraîne ton modèle sur X_train, puis crée une matrice de features X_train_CV. Crée la matrice X_test_CV sans ré-entraîner le modèle. Le format de la matrice X_test_CV doit être 4091x15806 avec 44633 stored elements.

In [ ]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(lowercase = True, stop_words = stopwords.words('english'))

In [ ]:
X_train_cv = vectorizer.fit_transform(X_train)
X_test_cv = vectorizer.transform(X_test)

In [ ]:
X_test_cv

<4091x15665 sparse matrix of type '<class 'numpy.int64'>'
	with 26312 stored elements in Compressed Sparse Row format>

In [ ]:
from sklearn.linear_model import LogisticRegression
tweet_model = LogisticRegression().fit(X_train_cv, y_train)

In [ ]:
print("score d'exactitude (accuracy) sur l'ensemble d'entraînement :", tweet_model.score(X_train_cv, y_train))
print("score d'exactitude (accuracy) sur l'ensemble de test :", tweet_model.score(X_test_cv, y_test))

score d'exactitude (accuracy) sur l'ensemble d'entraînement : 0.9526564537157758
score d'exactitude (accuracy) sur l'ensemble de test : 0.8995355658763139


# Étape bonus : essaye d'afficher 10 tweets qui ont été mal prédits (faux positifs ou faux négatifs). Aurais-tu fait mieux que l'algorithme ?

In [ ]:
from sklearn.metrics import confusion_matrix

pd.DataFrame(data = confusion_matrix(y_true = y_test, y_pred = tweet_model.predict(X_test_cv)),
             index = tweet_model.classes_ + " actual",
             columns = tweet_model.classes_ + " predicted")

,negative predicted,positive predicted
negative actual,1786,149
positive actual,262,1894


In [ ]:
X_pred = vectorizer.transform(X)

In [ ]:
df_pos_neg['predicted'] = tweet_model.predict(X_pred)

<ipython-input-61-ec010d39a42c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pos_neg['predicted'] = tweet_model.predict(X_pred)


In [ ]:
df_pos_neg

,textID,text,selected_text,sentiment,predicted
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,negative
2,088c60f138,my boss is bullying me...,bullying me,negative,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,negative
6,6e0c6d75b1,2am feedings for the baby are fun when he is a...,fun,positive,positive
...,...,...,...,...,...
27474,b78ec00df5,enjoy ur night,enjoy,positive,positive
27475,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative,negative
27476,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative,negative
27477,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive,positive


In [ ]:
df_pos_neg[df_pos_neg['sentiment'] != df_pos_neg['predicted']].head(10)

,textID,text,selected_text,sentiment,predicted
62,2a9765b7f9,I give in to easily,I give in to easily,negative,positive
122,66057d58c2,"Not looking forward to next week: Maths, Geogr...",Not looking forward,negative,positive
124,f0460d611d,not well,not well,negative,positive
125,6649f3558c,Not a prob hun,Not a prob,positive,negative
139,cb1b0d191d,"missed all the awesome weather, because she wa...","missed all the awesome weather,",negative,positive
148,ae71041e21,Candle wax is very enjoyable.,enjoyable.,positive,negative
175,079e4e3472,or even NOOOOO NOT THE SECRET NAMEREBECCA PLEASE,NOT THE SECRET NAMEREBECCA PLEASE,negative,positive
217,ca832cad51,Feeling smooth like chrome,Feeling smooth,positive,negative
240,7df7221145,that`s why I need to be there...To represent...,that`s why I need to be there,positive,negative
252,96a4ce0d47,almost died. Laptop screen was set to 100% bri...,died.,positive,negative
